### Phase 3 (Data Preparation)

In [1]:
# imports
import pandas as pd
import numpy as np

### Reading the 'combined_data.csv' Resulting from the Previous Phase into a Dataframe

In [2]:
# read data from storage
data = pd.read_csv('../data/combined_data.csv')
data.head()

,LCLid,date,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,temperatureMax,temperatureMin,windBearing,cloudCover,windSpeed,Type,stdorToU,Acorn,Acorn_grouped,file
0,MAC000131,2011-12-15,0.4850,0.432045,0.868,22,0.239146,9.505,0.072,7.97,4.08,234.0,0.42,4.71,No holiday,Std,ACORN-E,Affluent,block_22
1,MAC000131,2011-12-16,0.1415,0.296167,1.116,48,0.281471,14.216,0.031,4.68,1.80,315.0,0.70,3.71,No holiday,Std,ACORN-E,Affluent,block_22
2,MAC000131,2011-12-17,0.1015,0.189812,0.685,48,0.188405,9.111,0.064,5.35,0.24,285.0,0.37,3.99,No holiday,Std,ACORN-E,Affluent,block_22
3,MAC000131,2011-12-18,0.1140,0.218979,0.676,48,0.202919,10.511,0.065,5.49,-0.56,289.0,0.22,3.60,No holiday,Std,ACORN-E,Affluent,block_22
4,MAC000131,2011-12-19,0.1910,0.325979,0.788,48,0.259205,15.647,0.066,6.64,-0.84,235.0,0.47,2.70,No holiday,Std,ACORN-E,Affluent,block_22


### Removing Unnecessary Columns

In [3]:
# drop obvious unnecessary columns
columns_to_drop = ['LCLid','stdorToU','file']
df = data.drop(columns=columns_to_drop)
df.head()

print(df['Acorn_grouped'].unique())

['Affluent' 'Adversity' 'Comfortable' 'ACORN-U' 'ACORN-']


### Converting String-Type Columns to Numeric

- Rename the 'Type' column to 'isHoliday' and assign values of 0 or 1, where 1 indicates any holiday and 0 otherwise.
- Convert the 'Acorn' and 'Acorn_grouped' columns to numeric by mapping unique values to numerical values.
- Correct the values in the 'Acorn_grouped' column where 'ACORN-U' and 'ACORN-' represent 'Urban' and 'Rising' respectively, according to the ACORN documentation.

In [4]:
# transform features to numerical format
df['isHoliday'] = df['Type'].apply(lambda x: 0 if x == 'No holiday' else 1)
df['Acorn_grouped'] = df['Acorn_grouped'].apply(lambda x: 'Urban' if x == 'ACORN-U' else ('Rising' if x == 'ACORN-' else x))


acorn_codes, acorn_uniques = pd.factorize(df['Acorn'])
acorn_grouped_codes, acorn_grouped_uniques = pd.factorize(df['Acorn_grouped'])

acorn_map = {value: code for value, code in zip(acorn_uniques, range(len(acorn_uniques)))}
acorn_grp_map = {value: code for value, code in zip(acorn_grouped_uniques, range(len(acorn_grouped_uniques)))}

for unique_str, code in acorn_map.items():
    print(f'Str: {unique_str}, Code: {code}')
for unique_str, code in acorn_grp_map.items():
    print(f'Str: {unique_str}, Code: {code}')

df['acorn'] = acorn_codes
df['acorn_grouped'] = acorn_grouped_codes


columns_to_drop = ['Type', 'Acorn', 'Acorn_grouped']
df.drop(columns=columns_to_drop, inplace=True)
df.head()

Str: ACORN-E, Code: 0
Str: ACORN-N, Code: 1
Str: ACORN-H, Code: 2
Str: ACORN-P, Code: 3
Str: ACORN-F, Code: 4
Str: ACORN-K, Code: 5
Str: ACORN-Q, Code: 6
Str: ACORN-I, Code: 7
Str: ACORN-L, Code: 8
Str: ACORN-D, Code: 9
Str: ACORN-J, Code: 10
Str: ACORN-U, Code: 11
Str: ACORN-O, Code: 12
Str: ACORN-A, Code: 13
Str: ACORN-B, Code: 14
Str: ACORN-C, Code: 15
Str: ACORN-M, Code: 16
Str: ACORN-G, Code: 17
Str: ACORN-, Code: 18
Str: Affluent, Code: 0
Str: Adversity, Code: 1
Str: Comfortable, Code: 2
Str: Urban, Code: 3
Str: Rising, Code: 4


,date,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,temperatureMax,temperatureMin,windBearing,cloudCover,windSpeed,isHoliday,acorn,acorn_grouped
0,2011-12-15,0.4850,0.432045,0.868,22,0.239146,9.505,0.072,7.97,4.08,234.0,0.42,4.71,0,0,0
1,2011-12-16,0.1415,0.296167,1.116,48,0.281471,14.216,0.031,4.68,1.80,315.0,0.70,3.71,0,0,0
2,2011-12-17,0.1015,0.189812,0.685,48,0.188405,9.111,0.064,5.35,0.24,285.0,0.37,3.99,0,0,0
3,2011-12-18,0.1140,0.218979,0.676,48,0.202919,10.511,0.065,5.49,-0.56,289.0,0.22,3.60,0,0,0
4,2011-12-19,0.1910,0.325979,0.788,48,0.259205,15.647,0.066,6.64,-0.84,235.0,0.47,2.70,0,0,0


### Removing Rows Containing NaN Values

In [5]:
# drop nan values

print(len(df))
print(df.isna().sum())
df = df.dropna()
print(len(df))
print(df.isna().sum())

3458019
date                  0
energy_median        30
energy_mean          30
energy_max           30
energy_count          0
energy_std        11293
energy_sum           30
energy_min           30
temperatureMax        0
temperatureMin        0
windBearing           0
cloudCover         5099
windSpeed             0
isHoliday             0
acorn                 0
acorn_grouped         0
dtype: int64
3441632
date              0
energy_median     0
energy_mean       0
energy_max        0
energy_count      0
energy_std        0
energy_sum        0
energy_min        0
temperatureMax    0
temperatureMin    0
windBearing       0
cloudCover        0
windSpeed         0
isHoliday         0
acorn             0
acorn_grouped     0
dtype: int64


### Dropping Outlier Values Following the Scatter Plot from the Previous Phase

- We will limit the values to a specific range based on the scatter plot.

In [6]:
# drop outlying values
median_range = (0, 4)
mean_range = (0, 4)
max_range = (0, 8)
std_range = (0, 2.5)
sum_range = (0, 190)
min_range = (0, 3)

df = df[
    (df['energy_median'] >= median_range[0]) & (df['energy_median'] <= median_range[1]) &
    (df['energy_mean'] >= mean_range[0]) & (df['energy_mean'] <= mean_range[1]) &
    (df['energy_max'] >= max_range[0]) & (df['energy_max'] <= max_range[1]) &
    (df['energy_std'] >= std_range[0]) & (df['energy_std'] <= std_range[1]) &
    (df['energy_sum'] >= sum_range[0]) & (df['energy_sum'] <= sum_range[1]) &
    (df['energy_min'] >= min_range[0]) & (df['energy_min'] <= min_range[1])
]

print(len(df))

3441480


### Converting the 'date' Column into Cyclical

- Since the algorithm cannot directly handle 'datetime' values, let's approach it by considering months and weeks as forming a circle, where different values are represented as sine and cosine values.

- We'll add new columns for months and weekdays so that the algorithm interprets dates in a way that "Tuesday comes after Monday".

- For months, if January is represented as 0, then its value would be cos = 1, sin = 0. February would have a cosine value less than 1 and a sine value greater than 0, thus each month and day represents different values in the coordinate system.

In [7]:
# add cyclic transformation to 'date'
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.dayofweek  # Monday=0, Sunday=6
df['week_of_year'] = df['date'].dt.isocalendar().week

df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

dfs = df.sort_values(by=['acorn', 'date'])

dfs.head()

,date,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,temperatureMax,temperatureMin,...,acorn_grouped,year,month,day,day_of_week,week_of_year,month_sin,month_cos,day_of_week_sin,day_of_week_cos
604678,2011-11-23,0.1725,0.248708,0.798,24,0.193996,5.969,0.055,10.36,3.81,...,0,2011,11,23,2,47,-0.5,0.866025,0.974928,-0.222521
605496,2011-11-23,0.6765,0.904250,2.407,24,0.592275,21.702,0.261,10.36,3.81,...,0,2011,11,23,2,47,-0.5,0.866025,0.974928,-0.222521
2284511,2011-11-23,0.2070,0.330500,1.162,28,0.313498,9.254,0.059,10.36,3.81,...,0,2011,11,23,2,47,-0.5,0.866025,0.974928,-0.222521
604679,2011-11-24,0.1250,0.174875,0.611,48,0.130562,8.394,0.036,12.93,8.56,...,0,2011,11,24,3,47,-0.5,0.866025,0.433884,-0.900969
605497,2011-11-24,0.4855,0.619208,2.159,48,0.415436,29.722,0.217,12.93,8.56,...,0,2011,11,24,3,47,-0.5,0.866025,0.433884,-0.900969


### Using the 'energy_sum' Column as the Target Variable for Prediction

- We will remove all columns that are completely correlated with the 'energy_sum' column from the dataset.
- Additionally, since the information encoded in the 'date' column is already represented through other engineered features ('year', 'month', 'day', 'day_of_week', 'week_of_year', 'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos'), we will also remove the 'date' column.

In [8]:
# dropping columns that are likely redundant and derived from the chose target variable 'energy_sum'

derived_and_redundant_columns = ['energy_median', 'energy_mean', 'energy_max', 'energy_count', 'energy_std', 'energy_min','date']
dfs = dfs.drop(columns=derived_and_redundant_columns)

dfs.head()

,energy_sum,temperatureMax,temperatureMin,windBearing,cloudCover,windSpeed,isHoliday,acorn,acorn_grouped,year,month,day,day_of_week,week_of_year,month_sin,month_cos,day_of_week_sin,day_of_week_cos
604678,5.969,10.36,3.81,229.0,0.36,2.04,0,0,0,2011,11,23,2,47,-0.5,0.866025,0.974928,-0.222521
605496,21.702,10.36,3.81,229.0,0.36,2.04,0,0,0,2011,11,23,2,47,-0.5,0.866025,0.974928,-0.222521
2284511,9.254,10.36,3.81,229.0,0.36,2.04,0,0,0,2011,11,23,2,47,-0.5,0.866025,0.974928,-0.222521
604679,8.394,12.93,8.56,204.0,0.41,4.04,0,0,0,2011,11,24,3,47,-0.5,0.866025,0.433884,-0.900969
605497,29.722,12.93,8.56,204.0,0.41,4.04,0,0,0,2011,11,24,3,47,-0.5,0.866025,0.433884,-0.900969


### Using One-Hot Encoding for the 'acorn' and 'acorn_grouped' Columns
- Employing One-Hot Encoding is essential for the 'acorn' and 'acorn_grouped' columns to prevent the model from misinterpreting categorical values as ordinal or having a meaningful order. Without One-Hot Encoding, the model might mistakenly assume an increasing significance in the 'acorn' classification, potentially leading to misinterpretations.

**Note:**
- The 'isHoliday' column is already a binary column with values 0 or 1. Binary columns do not represent a similar ordinal order as categorical columns (which could have values like 0, 1, 2, 3, 4, 5, 6, etc.), hence, they do not require any encoding.

In [9]:
# apply onehot encoding to categorical variables before splitting
from sklearn.preprocessing import OneHotEncoder

one_hot = OneHotEncoder()

columns_to_encode = ['acorn', 'acorn_grouped']

encoded_data = one_hot.fit_transform(dfs[columns_to_encode])
encoded_data_dense = encoded_data.toarray()

encoded_columns = one_hot.get_feature_names_out(columns_to_encode)

encoded_df = pd.DataFrame(encoded_data_dense, columns=encoded_columns, index=dfs.index)

df_encoded = pd.concat([dfs.drop(columns_to_encode, axis=1), encoded_df], axis=1)

df_encoded.head()


,energy_sum,temperatureMax,temperatureMin,windBearing,cloudCover,windSpeed,isHoliday,year,month,day,...,acorn_14,acorn_15,acorn_16,acorn_17,acorn_18,acorn_grouped_0,acorn_grouped_1,acorn_grouped_2,acorn_grouped_3,acorn_grouped_4
604678,5.969,10.36,3.81,229.0,0.36,2.04,0,2011,11,23,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
605496,21.702,10.36,3.81,229.0,0.36,2.04,0,2011,11,23,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2284511,9.254,10.36,3.81,229.0,0.36,2.04,0,2011,11,23,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
604679,8.394,12.93,8.56,204.0,0.41,4.04,0,2011,11,24,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
605497,29.722,12.93,8.56,204.0,0.41,4.04,0,2011,11,24,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### Splitting the Data into Train and Test Sets
- It's crucial to perform this step before scaling to prevent data leakage. Data leakage occurs when information from the test set unintentionally influences the model during training, leading to overly optimistic performance metrics.
- Scaling should only be applied to the training set to maintain data integrity. If scaling is done before splitting the dataset, the scaling process may utilize information from the entire dataset, causing data leakage.
- Therefore, the scaling process should use only the values from the X_train set to ensure data integrity in both the training and test sets.
- Subsequently, X_test will be scaled using the parameters from X_train to ensure that the model has not seen any information or parameters from the test set at any stage.

In [10]:
# splitting data
from sklearn.model_selection import train_test_split

X = df_encoded.drop('energy_sum', axis=1)
y = df_encoded['energy_sum']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

2753184
2753184
688296
688296


### Scaling

After splitting the data, we will proceed with scaling:

- Initially, we'll use StandardScaler() with default parameters.
- We'll fit the scaler to the values of the X_train dataset for optimization.
- Then, we'll scale both the X_train and X_test sets using the fitted scaler.

In [16]:
# scaling
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

columns_to_scale = [
    'temperatureMax', 'temperatureMin', 'windBearing', 'cloudCover', 'windSpeed',
    'year', 'month', 'day', 'day_of_week', 'week_of_year'
]

#scaler = StandardScaler()
scaler = MinMaxScaler()
scaler.fit(X_train[columns_to_scale])


# Transform the training data
X_train_scaled = scaler.transform(X_train[columns_to_scale])
X_train.loc[:, columns_to_scale] = X_train_scaled

# Transform the test data
X_test_scaled = scaler.transform(X_test[columns_to_scale])
X_test.loc[:, columns_to_scale] = X_test_scaled

X_train

,temperatureMax,temperatureMin,windBearing,cloudCover,windSpeed,isHoliday,year,month,day,day_of_week,...,acorn_14,acorn_15,acorn_16,acorn_17,acorn_18,acorn_grouped_0,acorn_grouped_1,acorn_grouped_2,acorn_grouped_3,acorn_grouped_4
898999,0.261861,0.265088,0.587744,0.17,0.000000,0,1.000000,0.000000,0.633333,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1977945,0.504313,0.522918,0.671309,0.34,0.508197,0,0.666667,0.363636,0.100000,0.833333,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1837257,0.199014,0.294118,0.362117,0.03,0.128074,0,0.666667,1.000000,0.333333,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1918826,0.282193,0.430099,0.640669,0.32,0.367828,0,0.333333,1.000000,0.533333,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
24962,0.489526,0.548128,0.752089,0.17,0.323770,0,0.333333,0.727273,0.933333,0.833333,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297987,0.375539,0.588999,0.362117,0.67,0.131148,0,0.666667,0.363636,0.900000,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2425263,0.702403,0.718869,0.050139,0.50,0.252049,0,0.666667,0.636364,0.800000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
774810,0.256624,0.329641,0.576602,0.65,0.575820,0,0.666667,0.181818,0.466667,0.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
460312,0.537277,0.561879,0.629526,0.45,0.304303,0,0.333333,0.727273,0.633333,0.500000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [17]:
# save scaler instance for future use
import joblib
joblib.dump(scaler, '../scalers/X_scaler.save')

['../scalers/X_scaler.save']

### Saving All Datasets Used for Modeling to .csv Files

In [18]:
# save the different variations of train and test data to './docs/data/training_data'

X_train.to_csv('../data/training_data/train/X_train.csv', index=False)
X_test.to_csv('../data/training_data/test/X_test.csv', index=False)
y_train.to_csv('../data/training_data/train/y_train.csv', index=False)
y_test.to_csv('../data/training_data/test/y_test.csv', index=False)

### Preparing Function for Phase 4 Implementation

- This function will be relocated to Phase 4, where it will be utilized in model inputs to transform 'datetime' values into cyclical representations, as demonstrated above."

In [13]:
# this is a function to convert date_str used later. 

""" def transform_date(date_str):
    # Convert string to datetime
    date = pd.to_datetime(date_str)
    
    # Extract date components and create transformations
    features = {
        'year': date.year,
        'month': date.month,
        'day': date.day,
        'day_of_week': date.dayofweek,
        'week_of_year': date.isocalendar().week,
        'month_sin': np.sin(2 * np.pi * date.month / 12),
        'month_cos': np.cos(2 * np.pi * date.month / 12),
        'day_of_week_sin': np.sin(2 * np.pi * date.dayofweek / 7),
        'day_of_week_cos': np.cos(2 * np.pi * date.dayofweek / 7),
    }
    
    features_df = pd.DataFrame([features])
    
    return features_df """

" def transform_date(date_str):\n    # Convert string to datetime\n    date = pd.to_datetime(date_str)\n    \n    # Extract date components and create transformations\n    features = {\n        'year': date.year,\n        'month': date.month,\n        'day': date.day,\n        'day_of_week': date.dayofweek,\n        'week_of_year': date.isocalendar().week,\n        'month_sin': np.sin(2 * np.pi * date.month / 12),\n        'month_cos': np.cos(2 * np.pi * date.month / 12),\n        'day_of_week_sin': np.sin(2 * np.pi * date.dayofweek / 7),\n        'day_of_week_cos': np.cos(2 * np.pi * date.dayofweek / 7),\n    }\n    \n    features_df = pd.DataFrame([features])\n    \n    return features_df "

### Visualizing a Subset of the Prepared Dataset

- Given the complexity of the dataset, selecting an appropriate graphical method for visualization can be challenging. However, we will attempt to visualize a subset of the prepared dataset to gain insights.

#### PCA X_train

- Below is a visual representation of three principal components from the X_train dataset.
- Principal Component Analysis (PCA) itself does not directly tell us what the principal components are or what they represent, but it can help to observe underlying patterns, relationships, or clusters within the data.
- Principal components are linear combinations of the original variables, organized in order of explanatory power.

In [14]:
""" from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Assuming 'data' is your DataFrame
pca = PCA(n_components=3)  # Reduce to 2 dimensions for plotting
pca_result = pca.fit_transform(X_train)

# Creating a DataFrame for the PCA results
pca_df = pd.DataFrame(data = pca_result, columns = ['principal component 1', 'principal component 2','principal component 3'])

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')  # Set up for 3D plotting

# Scatter plot for 3D PCA results
ax.scatter(pca_df['principal component 1'], pca_df['principal component 2'], pca_df['principal component 3'], c='b', marker='o', alpha=0.5)

ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
plt.title('3D PCA on Dataset')

plt.show() """

" from sklearn.decomposition import PCA\nimport matplotlib.pyplot as plt\nfrom mpl_toolkits.mplot3d import Axes3D\n\n# Assuming 'data' is your DataFrame\npca = PCA(n_components=3)  # Reduce to 2 dimensions for plotting\npca_result = pca.fit_transform(X_train)\n\n# Creating a DataFrame for the PCA results\npca_df = pd.DataFrame(data = pca_result, columns = ['principal component 1', 'principal component 2','principal component 3'])\n\nfig = plt.figure(figsize=(10, 8))\nax = fig.add_subplot(111, projection='3d')  # Set up for 3D plotting\n\n# Scatter plot for 3D PCA results\nax.scatter(pca_df['principal component 1'], pca_df['principal component 2'], pca_df['principal component 3'], c='b', marker='o', alpha=0.5)\n\nax.set_xlabel('Principal Component 1')\nax.set_ylabel('Principal Component 2')\nax.set_zlabel('Principal Component 3')\nplt.title('3D PCA on Dataset')\n\nplt.show() "